In [2]:
import os
from dotenv import load_dotenv
load_dotenv("/Users/gkreder/bioimageio-chatbot/.vscode/.env")
from bioimageio_chatbot.chatbot import create_customer_service, QuestionWithHistory, UserProfile
from schema_agents.schema import Message
import asyncio
import yaml
import pytest
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from bioimageio_chatbot.knowledge_base import create_vector_knowledge_base
from bioimageio_chatbot.chatbot import load_model_info

2023-11-28 11:06:17.994 | INFO     | schema_agents.config:__init__:44 - Config loading done.
2023-11-28 11:06:17.995 | INFO     | schema_agents.config:__init__:55 - Set OPENAI_API_BASE in case of network issues


In [58]:

models = load_model_info()
# models = [m for m in models if 'nickname' in m]
# rdf_sources = [m['rdf_source'] for m in model_info]
# nicknames = [m['nickname'] for m in model_info]

In [60]:
m['id'].replace('/', '-')

'10.5281-zenodo.8420099'

In [138]:
import requests
from tqdm.auto import tqdm
import yaml
import sys
import re

yaml_dir = "/Users/gkreder/Downloads/2023-11-28_yamls"
os.makedirs(yaml_dir, exist_ok=True)

models_out =[]
for m in tqdm(models):
    response = requests.get(m['rdf_source'])
    if response.status_code == 200:
        content = response.content.decode('utf-8')
        yaml_dict = yaml.safe_load(content)
        test_inputs = re.findall(r'test_inputs:(.*)', content)
        nicknames = re.findall(r'nickname:(.*)', content)
        if len(test_inputs) * len(nicknames) == 0:
            continue
        nickname = nicknames[0].strip()
        test_inputs = yaml_dict['test_inputs']
        file_name_base = f"{nickname}.yaml"
        file_name = os.path.join(yaml_dir, file_name_base)
        with open(file_name, 'w') as f:
            f.write(content)

100%|██████████| 141/141 [00:03<00:00, 43.52it/s]


In [220]:
import torch
from torchvision import models, transforms
from PIL import Image
import torch.nn as nn

resnet50_model = models.resnet50(pretrained=True)
resnet50_model.avgpool = nn.AdaptiveAvgPool2d((1, 1))
resnet50_model = nn.Sequential(*list(resnet50_model.children())[:-1])  # Removing the last fully connected layer
resnet50_model.eval()



# data_dir = "/Users/gkreder/Downloads/2023-11-28_data/"
# all_metadata = []
# all_embeddings = []
# os.makedirs(data_dir, exist_ok=True)
# for fname in tqdm([x for x in os.listdir(yaml_dir) if x.endswith('.yaml')]):
#     fname_abs = os.path.join(yaml_dir, fname)
#     with open(fname_abs, 'r') as f:
#         content = f.read()
#         yaml_dict = yaml.safe_load(content)
#     test_inputs = yaml_dict['test_inputs']
#     for i_ti, ti in enumerate(test_inputs):
#         response = requests.get(ti)
#         if response.status_code == 200:
#             data_file = os.path.join(data_dir, fname.replace('.yaml', f'_{i_ti}.npy'))
#             with open(data_file, 'wb') as file:
#                 file.write(response.content)
#         embedding = get_embedding(data_file, vgg16_model, vgg16_preprocess)
#         all_embeddings.append(embedding)
#         all_metadata.append(yaml_dict)


/Users/gkreder/miniconda3/envs/novel-designer/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/gkreder/miniconda3/envs/novel-designer/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /Users/gkreder/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:04<00:00, 25.6MB/s]


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [222]:
# np.load(data_file).shape
torch_tens = torch.from_numpy(np.load(data_file))
with torch.no_grad():
        embedding = resnet50_model(torch_tens)

RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[1, 576, 720, 1] to have 3 channels, but got 576 channels instead

In [ ]:
resnet50_model()

In [150]:
import numpy as np

np.load("/Users/gkreder/Downloads/test_input_0.npy").shape

(1, 1, 512, 512)

In [206]:
import torch
from torchvision import models, transforms
from PIL import Image

vgg16_model = models.vgg16(pretrained=True)
vgg16_model.eval()
vgg16_model.classifier = vgg16_model.classifier[:-1]

# Define the image transformation
vgg16_preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def npy_to_tensor(npy_path, preprocess):
    # Load .npy file
    npy_image = np.load(npy_path)

    # Handle 2D images (grayscale)
    if len(npy_image.shape) == 2:
        # Convert grayscale to RGB by duplicating the channels
        npy_image = np.stack((npy_image,) * 3, axis=-1)

    # Handle additional dimensions (e.g., take the first slice or mean along the z-axis)
    while len(npy_image.shape) > 3:
        # This is a simple approach, you might need to adjust this based on your specific case
        npy_image = npy_image.mean(axis=2)  # Example: mean along the z-axis

    # Convert to PIL Image for preprocessing
    img = Image.fromarray(npy_image.astype('uint8'), 'RGB')
    img_t = preprocess(img)
    batch_t = torch.unsqueeze(img_t, 0)

    return batch_t

def get_embedding(npy_path, model, preprocess):
    tensor = npy_to_tensor(npy_path, preprocess)
    with torch.no_grad():
        embedding = model(tensor)
    return embedding




/Users/gkreder/miniconda3/envs/novel-designer/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/gkreder/miniconda3/envs/novel-designer/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [181]:
# Image.open("/Users/gkreder/Downloads/cat.jpeg")
# embedding = image_to_embedding("/Users/gkreder/Downloads/cat.jpeg", vgg16_model, preprocess)

npy_path = "/Users/gkreder/Downloads/test_input_0.npy"
npy_image = np.load(npy_path)
embedding = get_embedding(npy_path, vgg16_model, vgg16_preprocess)

In [184]:
embedding.sum()

tensor(406.9442)

In [185]:
import pickle as pkl

In [223]:
import torch
from torchvision import models, transforms
from PIL import Image
import numpy as np

# Load a pre-trained ResNet model with adaptive pooling
model = models.resnet50(pretrained=True)
model.avgpool = torch.nn.AdaptiveAvgPool2d((1, 1))
model = torch.nn.Sequential(*list(model.children())[:-1])
model.eval()

# Define a generic image transformation
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def npy_to_embedding(npy_path, model, preprocess):
    # Load the .npy file
    npy_image = np.load(npy_path)

    # Handle different shapes
    # Convert 2D (grayscale) to 3D (RGB)
    if len(npy_image.shape) == 2:
        npy_image = np.stack((npy_image,) * 3, axis=-1)
    # Assuming the array is in HxWxC format, if not, you might need to transpose

    # Convert to PIL Image for preprocessing (handle uint8 and floating point)
    if npy_image.dtype == np.float32 or npy_image.dtype == np.float64:
        # Scale to 0-255 if it's not already
        npy_image = (npy_image - npy_image.min()) / (npy_image.max() - npy_image.min())
        npy_image = (npy_image * 255).astype('uint8')
    img = Image.fromarray(npy_image)

    # Apply preprocessing
    img_t = preprocess(img)
    batch_t = torch.unsqueeze(img_t, 0)

    with torch.no_grad():
        embedding = model(batch_t)

    return embedding.flatten()

# Example usage
npy_path = data_file
embedding = npy_to_embedding(npy_path, model, preprocess)


/Users/gkreder/miniconda3/envs/novel-designer/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/gkreder/miniconda3/envs/novel-designer/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


TypeError: Cannot handle this data type: (1, 1, 720, 1), |u1